In [1]:
import pre_processing as pp
from sklearn.ensemble import RandomForestClassifier

In [5]:
files = pp.get_files("/Users/azamkhan/github/pdiot-practical/pdiot-data/2020/")
data = pp.get_data("Chest_Left", files)

In [6]:
data, tgts, students = pp.get_data_by_datapoints(data, points_wanted = 50)
tgts = pp.convert_targets(tgts)

In [7]:
cleaned_data, targets, student_data, student_targets = pp.leave_one_out(data, tgts, students)

In [188]:
clf = RandomForestClassifier()
clf.fit(cleaned_data, targets)
score = clf.score(student_data, student_targets)
print(score)

0.6795580110497238


In [189]:
timestep = 50
x_train = np.array([[c[0:timestep], c[timestep:2*timestep], c[2*timestep:3*timestep]] for c in cleaned_data])
x_test = np.array([[c[0:timestep], c[timestep:2*timestep], c[2*timestep:3*timestep]] for c in student_data])
x_train.shape

(8917, 3, 50)

In [190]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],x_train.shape[2],1)

In [191]:
x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],x_test.shape[2],1)

In [192]:
y_train = targets
y_test = student_targets

In [193]:
x_train.shape

(8917, 3, 50, 1)

In [208]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D, Conv1D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow import keras

import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

model = Sequential()
model.add(Conv2D(128, (2,2), activation='relu', input_shape = x_train[0].shape))
model.add(Dropout(0.2))

model.add(Conv2D(128, (2,2), activation='relu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=1))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(14, activation='softmax'))

In [209]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

In [210]:
history = model.fit(x_train, y_train, epochs=20, batch_size = 16, validation_data = (x_test, y_test), verbose=1)

Epoch 1/20
558/558 [==============================] - 7s 13ms/step - loss: 1.5735 - accuracy: 0.4729 - val_loss: 0.7529 - val_accuracy: 0.7315
Epoch 2/20
558/558 [==============================] - 7s 13ms/step - loss: 1.1442 - accuracy: 0.5871 - val_loss: 0.5569 - val_accuracy: 0.8188
Epoch 3/20
558/558 [==============================] - 7s 13ms/step - loss: 1.0329 - accuracy: 0.6168 - val_loss: 0.5020 - val_accuracy: 0.8376
Epoch 4/20
558/558 [==============================] - 7s 13ms/step - loss: 0.9745 - accuracy: 0.6341 - val_loss: 0.7358 - val_accuracy: 0.6928
Epoch 5/20
558/558 [==============================] - 7s 13ms/step - loss: 0.9259 - accuracy: 0.6461 - val_loss: 0.5107 - val_accuracy: 0.8729
Epoch 6/20
558/558 [==============================] - 8s 14ms/step - loss: 0.8743 - accuracy: 0.6572 - val_loss: 0.5544 - val_accuracy: 0.8110
Epoch 7/20
558/558 [==============================] - 7s 13ms/step - loss: 0.8469 - accuracy: 0.6784 - val_loss: 1.0840 - val_accuracy: 0.7215